# Marc tools

## Import

In [ ]:
import os
import plotly
import importlib
import re
import h5py

import numpy as np
import pandas as pd
import plotly.io as pio
import plotly.graph_objs as go
import marc_tools as mt
import plotly.express as px

from marc_tools.general import ss
from scipy.interpolate import interp1d
from pathlib import Path
from IPython.display import clear_output

# Set up plotly
plotly.offline.init_notebook_mode()

pio.templates['custom'] = go.layout.Template(
    layout_paper_bgcolor='rgba(0,0,0,0)',
    layout_plot_bgcolor='rgba(0,0,0,0)',
    ) 

# Change background color if using VSC
if 'VSCODE_CWD' in os.environ:
    pio.templates.default = 'plotly+custom'

# Pandas decimal precision
pd.set_option("display.precision", 4)

## BFLEX

### Stress

In [ ]:
# Define folders
project_dir = Path(r'C:\Projects\1993_Pierce_GI_riser')
bflex_dir = project_dir / 'Bflex/Local/Top_300bar'
df_bflex = pd.DataFrame()


# Find curvature
for curv in bflex_dir.glob(r'*elcurvY.mpf'):
    df_curv = mt.bflex.mpf_reader(curv)

df_curv.columns = ['curv']

# Find mpf_files
df_mpf = pd.DataFrame()

for mpf in bflex_dir.glob(r'*nosl?.mpf'):
    
    df_temp = mt.bflex.mpf_reader(mpf)
    df_mpf = pd.concat([df_mpf, df_temp], axis=1)

# Manipulate df
df_temp = pd.DataFrame()

for i in range(17, 33):
    df_temp[f'P{i}'] = df_mpf.filter(regex=f'{i}').max(axis=1)

df_stress = pd.concat([df_curv, df_temp], axis=1)

# Interpolate curvature
df_curv = pd.DataFrame({'curv' : [i/100 for i in range(0, 21)]})

for col in df_stress.columns[1:]:
    df_curv[col] = np.interp(df_curv.curv, df_stress.curv, df_stress[col])

df_bflex = df_curv

# Dictionary translating degrees to bflex position name
bflex_name = {}
for i in range(0, 16, 1):

    #  Establish relationship between theta and pos
    theta = 270 - 22.5 * i
    pos = f'P{17 + i}'
    
    # Only positve numbers beteen 0 and 360
    if theta < 0:
        theta += 360
    if theta > 360:
        theta -= 360
    
    bflex_name[pos] = theta

### Plotting

In [ ]:
# Plotting
fig_bf = go.Figure()

# Add first column to plot complete circle
df_plot = df_bflex.copy()
df_plot['P33'] = df_plot['P17']

# Plot every n'th increment
for inc in range(0, 15, 5):
    fig_bf.add_trace(go.Scatterpolar(r=df_plot.iloc[inc, 1:],
                                     theta=[-90 + 22.5*i for i in range(0, 18, 1)],
                                     fill='toself',
                                     name=f'Curv {df_plot.curv.loc[inc]}'
                                     ))



# Update layout of figure
fig_bf.update_layout(
    title=f'Stress BFLEX',
    autosize=False,
    font=dict(color='white'),
    width=1000, height=800,
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, 500]
        )),
    showlegend=True
)

fig_bf.show()

## Orcaflex

In [ ]:
# Read orcaflex files
orca_curv = pd.read_csv(r'curvature.csv')
orca_pos = pd.read_csv(r'elementpos.csv', index_col=[0])
orca_disp = pd.read_csv(r'motions.csv', sep=';', index_col=[0])

# Convert units from [m] to [mm]
orca_pos = orca_pos * 1000# -> mm
orca_disp.y = orca_disp.y*1000  # -> mm

df_resp_orca = orca_disp


# Load marc displacements and rotations
df_resp_marc = pd.read_csv(r'marc_response.csv', sep=';', index_col=None)
df_resp_marc.rx = np.degrees(df_resp_marc.rx)  # -> deg
df_resp_marc = df_resp_marc.where(df_resp_marc.inc>=10).dropna()

## Manipulation of orcaflex df
orca_curv = pd.concat([orca_curv, orca_disp], axis=1)

df_orca = pd.DataFrame()

for col in orca_curv.columns[1:len(orca_pos)]:
    df_col = orca_curv[['inc', col, 'y', 'rx']].rename({col:'curv'}, axis=1)
    df_col['z'] = orca_pos.arclength.loc[int(col)]
    df_orca = pd.concat([df_orca, df_col], axis=0).reset_index(drop=True)

df_orca

# Dictionary translating degrees to bflex position name
bflex_name = {}
for i in range(0, 16, 1):

    #  Establish relationship between theta and pos
    theta = 270 - 22.5 * i
    pos = f'P{17 + i}'
    
    # Only positve numbers beteen 0 and 360
    if theta < 0:
        theta += 360
    if theta > 360:
        theta -= 360
    
    bflex_name[pos] = theta

# Loop through df_orca
df_temp = pd.DataFrame()

for idx in df_orca.index:
    dfi = df_orca[df_orca.index==idx]
    dfi.curv

    # For each line, find nearest 2 rows with curv for this curvature (df1)
    df_interp = df_bflex.iloc[(df_bflex.curv - float(dfi.curv)).abs().argsort()[:2]]

    # Interpolate the curvature to get stress
    y1, y2 = df_interp.iloc[0,1:], df_interp.iloc[1,1:]
    stress = interp1d(df_interp.curv, np.vstack([y1, y2]), axis=0)(dfi.curv)[0]

    # Duplicate idx df to have same number of rows as stress
    dfi = dfi.loc[dfi.index.repeat(len(stress))]

    # Find circum position (using same relative orientation)
    # (sort bflex names on keys) CHECK
    theta = dict(sorted(bflex_name.items(), key=lambda item: item[0])).values()

    # Create dataframe (df2) where columns are stress for each circumferential position - rembember bflex_name
    dfi['stress'] = stress
    dfi['theta'] = theta
    dfi.sort_values('theta', inplace=True)
    df_temp = pd.concat([df_temp, dfi], axis=0)


df_orca = df_temp.reset_index(drop=True)

In [ ]:
ss(df_orca.z)

### Marc

#### Stress

In [ ]:
# Define marc files
marc_dir = Path(r'\\nas-ask-001\Projects\1993_Pierce_GIR_fatigue\Marc\jupyter_notebook')
marc_files = marc_dir.glob('*stresses_inner_ta_inc*[0-9].csv*')

# Bad incs which are to be skipped
bad_incs = [26, 28, 29, 38, 39, 40, 46, 47, 51, 57, 64, 65, 67]

# Read list with coordinates per node
df_list = pd.read_csv(marc_dir / 'angles.csv', delimiter='\t').iloc[:,1:]
df_list.drop('node_index', axis=1, inplace=True)

df = pd.DataFrame()

# Iterate over files and create dataframe
for mf in marc_files:

    # Read increment
    inc = int(re.findall(r'\d+', mf.stem)[0])

    # Skip bad increments
    if inc in bad_incs:
        continue
    
    # Read stress dataframe and add x, y and z coordinate
    df_read = pd.read_csv(mf, index_col=None).iloc[:,1:]

    # If node names are equal in both data frames, coordinates can be added
    if all(df_read['node'] == df_list['node_id']):
        df_read = pd.concat([df_read, df_list[['z', 'theta']]], axis=1)

    # Concat each stress file (one for each increment)
    df = pd.concat([df, df_read], axis=0)

df.z = np.round(df.z, 1)

df.sort_values(['inc', 'z', 'theta'], inplace=True)


In [ ]:
# Initialize dataframe containing marc information
df_marc = pd.DataFrame()

# Loop through each increments
increments = sorted(set(df.inc))[10:]
for inc in increments:
    print(f'{inc/70*100:.0f} %')

    # Skip bad increments
    if inc in bad_incs:
        continue

    # Create dataframe only containing 'inc'
    df_inc = df[df.inc==inc]

    # Combine groups where difference in z positions is < 10 - check sort on z
    df_inc['diff_z'] = df_inc.groupby(['inc'])['z'].diff().gt(10).cumsum()

    # Sort dataframe based on diff_z and theta to allow .diff to work
    df_inc.sort_values(['diff_z', 'theta'], inplace=True)

    # Combine groups where difference in theta is < 3
    df_inc['diff_th'] = df_inc.groupby(['inc', 'diff_z'])['theta'].diff().gt(3).cumsum()

    # Sort by diff and take max for each group
    dfg = df_inc.groupby(['diff_z', 'diff_th']).max().reset_index()
    # dfg.reset_index(inplace=True)

    # Give each position group (diff_z) same coordinates
    # can it be better than looping through each position?
    for p in sorted(set(dfg.diff_z)):
        dfg.loc[(dfg.diff_z==p), 'z'] = np.average(list(set(dfg[dfg.diff_z==p].z)))

    dfg.drop(columns=['diff_z', 'diff_th'], inplace=True)

    # Append dataframe for this inc to total dataframe
    df_marc = pd.concat([df_marc, dfg], axis=0)
    clear_output(wait=True)

# Round z
df_marc['z'] = [int(i) for i in np.round(list(df_marc['z']))]
df_marc['inc'] = [int(i) for i in df_marc['inc']]
df_marc['node'] = [int(i) for i in df_marc['node']]

# Space saving
df_marc = df_marc.astype({'theta': 'float32', 'stress': 'float32'})
clear_output()

# Remove unnceessary info from marc df
df_marc = df_marc.where(df_marc['inc']>=10).dropna()
df_marc.drop('node', axis=1, inplace=True)

## CrossSection

### Read files

In [ ]:
# Define locations
case_name = r'nov_8inch_gir_3p_with_bs_g3_job1'
project_folder = Path(r'V:\1993_Pierce_GIR_fatigue\Marc\analysis')

# Function specific input
runs = ['CS'] # Folder with runs
exclude_cases = [] # optional - cases to be excluded from post-processing

# Read all the cross section files
df_case = mt.crossection.df_cs(project_folder, runs, positions='start')

# Marc list
marc_list = sorted(list(set(df_marc.z)))

# Cross section list
cs_list = [int(k) for k in df_case[case_name].keys()]

# Find closest values
cs_dict = {}
for cs in cs_list:
    cs_dict[cs] = min(marc_list, key=lambda x: abs(x - cs))

# Add column
df_marc['curv'] = np.nan

# Reduce dataframe to points where CS data is available
for i, (cs_pos, marc_pos) in enumerate(cs_dict.items(), 1):
    print(f'{i/len(cs_dict)*100:.0f} %')
    
    for inc in range(0, 71, 1):
        df_marc.loc[(df_marc.z==marc_pos) & (df_marc.inc==inc), 'curv']\
        = df_case[case_name][str(cs_pos)]['K3_4'][inc]

    clear_output(wait=True)

# Remove rows where curv is na
df_marc = df_marc[df_marc.curv.notna()]
clear_output()

### Plotting

In [ ]:
# Position to plot
pos = 4
marc_pos = sorted(list(set(df_marc.z)))

fig_marc = go.Figure()

# Plot every n'th increment
increments = sorted(set(df_marc.inc))
for inc in increments:
    
    # Skip bad increments
    while inc in bad_incs:
        inc += 1

    # Only plot relevant and duplicate frist row last to close loop
    df_plot = df_marc[(df_marc.inc==inc) & (df_marc.z==marc_pos[pos])]

    # Find curvature
    curv = df_plot.curv.iloc[0]

    # Drop every n'th row (for plotting purposes)
    df_plot = df_plot.iloc[::3]
    df_plot = df_plot.append(df_plot.reset_index(drop=True).iloc[0]).reset_index(drop=True)

    # Plot polar plot
    fig_marc.add_trace(go.Scatterpolar(r=df_plot['stress'],
                                  theta=df_plot['theta'],
                                  fill='toself',
                                  name=f'Curvature {curv:.2f} (inc: {inc})'
                                  ))

# Update layout of figure
fig_marc.update_layout(
    title=f'Stress Marc @ {marc_pos[pos]}',
    autosize=False,
    font=dict(color='white'),
    width=1000, height=800,
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, 500]
        )),
    showlegend=True
)

fig_marc.show()

### Calculate SCF

In [ ]:
# Initialize data frame
df_scf = pd.DataFrame()
badinc = []

# Find axial positions (based on orcaflex)
orca_pos = sorted(list(set(df_orca.z)))
marc_pos = sorted(list(set(df_marc.z)))

# For each orca position
for i, pos in enumerate(orca_pos, 1):
    print(f'{i/len(orca_pos):.0%}')

    # Skip if outside boundary of marc
    if pos > marc_pos[-1]:
        continue

    # Find nearest marc positions
    df_pos = df_marc[df_marc.inc==30].groupby('z').max().reset_index()
    near_pos = mt.general.find_nearest(df_pos.z, pos)

    # Find corresponding dataframes for later interpolation
    df1 = df_marc[df_marc.z==near_pos[0]]
    df2 = df_marc[df_marc.z==near_pos[1]]

    # For each bflex angle, find nearest marc angle position
    theta_dict1, theta_dict2 = {}, {}
    

    for bf_angle in [22.5*i for i in range(0, 16, 1)]:
        theta_dict1[bf_angle] = min(df1.theta, key=lambda x: abs(x - bf_angle))
        theta_dict2[bf_angle] = min(df2.theta, key=lambda x: abs(x - bf_angle))

    # For each inc
    for inc in list(set(df_orca.inc)):
        
        # Create temp dataframe with orca data
        df_temp = df_orca[(df_orca.z==pos) & (df_orca.inc==inc)]

        # Interpolate to correct marc inc..

        # Find nearest marc inc
        # HERE IS WHERE TO CHANGE TO COMPARE WITH DISP OR ROTATION
        near_inc = df_resp_marc.iloc[(df_resp_marc.rx - df_resp_orca.rx.loc[inc]).abs().argsort().reset_index(drop=True)[0]].inc

        # Skip bad marc increments
        if near_inc in bad_incs:
            badinc.append(inc)
            continue

        # Simplify dataframe
        dft1 = df1[df1.inc==near_inc]
        dft2 = df2[df2.inc==near_inc]

        # Find marc stress for correct curvature and angle
        dft1 = dft1[dft1.theta.isin(theta_dict1.values())]
        dft2 = dft2[dft2.theta.isin(theta_dict2.values())]

        # Add bflex angle
        dft1['theta_bflex'] = theta_dict1.keys()
        dft2['theta_bflex'] = theta_dict2.keys()

        # Sort values to ensure stress in correct order
        dft1.sort_values('theta_bflex', inplace=True)
        dft2.sort_values('theta_bflex', inplace=True)

        # Interpolate marc based on position
        df_temp['stress_marc'] = interp1d(near_pos, np.vstack([dft1.stress, dft2.stress]), axis=0)(pos)
        df_temp['theta_marc'] = interp1d(near_pos, np.vstack([dft1.theta, dft2.theta]), axis=0)(pos)
        df_temp['rx_marc'] = float(df_resp_marc[df_resp_marc.inc==near_inc].rx)

        # Calculate scf
        df_temp['scf'] = df_temp.stress_marc / df_temp.stress

        # Concat
        df_scf = pd.concat([df_scf, df_temp.reset_index(drop=True)], axis=0)
        clear_output(wait=True)

# Round
df_scf.z = np.round(df_scf.z)

clear_output()

In [ ]:
# Plotting
fig_scf = go.Figure()
positions = sorted(list(set(df_scf.z)))
pos = 1

increments = sorted(set(df_scf.inc))[::9]

for inc in increments:
    
    # Extrat specific positinos
    df_plot = df_scf[(df_scf.inc==inc) & (df_scf.z==positions[pos])]
    df_plot = df_plot.append(df_plot.iloc[0]).reset_index()

    # Find curvature
    curv = df_plot.curv.iloc[0]

    # Plot
    fig_scf.add_trace(go.Scatterpolar(r=df_plot['scf'],
                                  theta=df_plot['theta'],
                                  fill='toself',
                                  name=f'Curvature {curv:.2f} (inc {inc})'
                                  ))

fig_scf.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, 1.3]
        )),
    showlegend=True
)

fig_scf.update_layout(title=f'Stress correction factor @ {positions[pos]:.0f} mm',
                      autosize=False,
                      font=dict(color='white'),
                      width=1000, height=800)

fig_scf.show()

In [ ]:
# Plot
df_max = df_scf.groupby(['inc', 'z']).max().reset_index()
df_max['scf_max'] = df_max.stress_marc / df_max.stress
inc = 50
data = []

# Max SCF
trace1 = go.Scatter(x=df_max[df_max.inc==inc]['z'],
                   y=df_max[df_max.inc==inc]['scf'],
                   name=f'Max SCF (circumf dependent)',
                   mode='lines+markers')

# Max stress marc / max stress bflex
trace2 = go.Scatter(x=df_max[df_max.inc==inc]['z'],
                    y=df_max[df_max.inc==inc]['scf_max'],
                    name=f'Max SCF (circumf INdependent)',
                    mode='lines+markers')

layout=go.Layout(title=f'Stress correction along length',
                 font=dict(color='white'),
                 xaxis=dict(title='Length [mm]'),
                 yaxis=dict(title='SCF [-]',
                            range=[0, 1.5]),
                 width=2000, height=800)

plotly.offline.iplot(go.Figure(data=[trace1, trace2], layout=layout))

In [ ]:
# Plot
df_max = df_scf.groupby(['inc', 'z']).max().reset_index()
df_max['scf_max'] = df_max.stress_marc / df_max.stress
inc = 50
data = []

# Create figure
fig = go.Figure()

# Max SCF
trace1 = go.Scatter(x=df_max[df_max.inc==inc]['z'],
                   y=df_max[df_max.inc==inc]['scf'],
                   name=f'Max SCF (circumf dependent)',
                   mode='lines+markers')

# Add traces, one for each increment
increments = ss(df_scf.inc)
for inc in increments:
    fig.add_trace(
        go.Scatter(
            visible=False,
            name=f"Max SCF {inc}",
            x=df_max[df_max.inc==inc]['z'],
            y=df_max[df_max.inc==inc]['scf']))

# Make 10th trace visible
fig.data[10].visible = True

# Create and add slider
steps = []
for i in range(len(fig.data)):
    inc = increments[i]
    end_rot = max(df_max[df_max.inc==inc]['rx'])
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)},
              {"title": f'Stress correction along length (end rotation {end_rot:.1f} deg)'}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)
    # try:
    #     fig['layout']['sliders'][0]['steps'][i]['label']=inc
    # except IndexError:
    #     pass

sliders = [dict(
    active=10,
    currentvalue={"prefix": "Increment: "},
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(
    sliders=sliders
)

for i, inc in enumerate(increments, start = 0):
    fig['layout']['sliders'][0]['steps'][i]['label']=inc

fig.update_layout(title=f'Stress correction along length',
                 font=dict(color='white'),
                 xaxis=dict(title='Length [mm]'),
                 yaxis=dict(title='SCF [-]',
                            range=[0, 1.5]),
                 width=2000, height=800)

fig.show()

## Curvature comparison

In [ ]:
# Plot
marc_inc = 50
orca_inc = 59
df_plot = df_scf[(df_scf.inc==marc_inc)]

# Calculate angles
marc_angle = np.degrees(np.average(df_scf[(df_scf.inc==marc_inc)].groupby(['z']).max().reset_index()['curv'])*4.95)
orca_angle = np.degrees(np.average(df_orca.loc[orca_inc])*5.48)

# Marc curvature
trace1 = go.Scatter(x=df_scf[(df_scf.inc==marc_inc)].groupby(['z']).max().reset_index()['z'],
                    y=df_scf[(df_scf.inc==marc_inc)].groupby(['z']).max().reset_index()['curv'],
                    name=f'Marc (y={df_resp_marc.y.loc[marc_inc]:.1f},<br>'
                         f'rx={df_resp_marc.rx.loc[marc_inc]:.1f})',
                    mode='lines+markers')

# Orcaflex curvature
trace2 = go.Scatter(x=orca_pos.arclength,
                    y=df_orca.loc[orca_inc],
                    name=f'Orcaflex (y={orca_disp.y.loc[orca_inc]:.1f},<br>'
                         f'rx={orca_disp.rx.loc[orca_inc]:.1f})',
                    mode='lines+markers')

layout=go.Layout(title=f'Curvature distribution',
                 font=dict(color='white'),
                 xaxis=dict(title='Length [mm]'),
                 yaxis=dict(title='Curvature [1/m]'),
                         #    range=[0, 1.5]),
                 width=2000, height=800)

plotly.offline.iplot(go.Figure(data=[trace1, trace2], layout=layout))

## Examples

In [ ]:
# Plotly 3d surface with color - Example
df_plot = df_marc.where(df_marc['inc']==60).dropna()

df_plot['x'] = 200.0 * np.sin(df_plot['theta'])
df_plot['y'] = 200.0 * np.cos(df_plot['theta'])

fig = go.Figure()

fig.add_trace(go.Surface(
    x=df_plot['x'].tolist()[0:100],
    y=df_plot['y'].tolist()[0:100],
    z=[df_plot['z'].tolist()[0:100]]*100,
    surfacecolor=df_plot['stress'].tolist()[0:10]
))
fig.update_layout(title='Stress', autosize=True,
                  width=1000, height=800)

fig.update_layout(scene=dict(xaxis=dict(range=[-200, 200]),
                             yaxis=dict(range=[-200, 200]),
                             zaxis=dict(range=[0, 4000])
))
fig.show()

In [ ]:
# Plotly express
fig = px.line_polar(df_marc, r='stress', theta='theta', line_close=True)
fig.update_traces(fill='toself')
fig.show()

## Read marc input

In [ ]:
a = 10

b = 2* 10